### Install GPT

In [1]:
import sys
#package_dir = "../input/ppbert/pytorch-pretrained-bert/pytorch-pretrained-BERT"
#sys.path.append(package_dir)
import os

os.listdir('../input')
! pip install ../input/gptpytorchoriginal/gpt2-pytorch/pytorch-pretrained-BERT-master

Processing /kaggle/input/gptpytorchoriginal/gpt2-pytorch/pytorch-pretrained-BERT-master
  Stored in directory: /tmp/.cache/pip/wheels/06/a8/0f/ffe000aebc5034e7a616a8c0292ff34e45ea66acfc168a0760
Successfully built pytorch-pretrained-bert
ERROR: allennlp 0.8.4 requires awscli>=1.11.91, which is not installed.
ERROR: allennlp 0.8.4 requires flaky, which is not installed.
ERROR: allennlp 0.8.4 requires responses>=0.7, which is not installed.
  Found existing installation: pytorch-pretrained-bert 0.6.2
    Uninstalling pytorch-pretrained-bert-0.6.2:
      Successfully uninstalled pytorch-pretrained-bert-0.6.2


### Imports

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

%reload_ext autoreload
%autoreload 2
%matplotlib inline

import fastai
from fastai.train import Learner
from fastai.train import DataBunch
from fastai.callbacks import *
from fastai.basic_data import DatasetType
import fastprogress
from fastprogress import force_console_behavior
import numpy as np
from pprint import pprint
import pandas as pd
import os
import time
import gc
import random
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
from keras.preprocessing import text, sequence
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F

import torch.utils.data
from tqdm import tqdm
import warnings
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam
from pytorch_pretrained_bert import BertConfig
from nltk.tokenize.treebank import TreebankWordTokenizer

from gensim.models import KeyedVectors

Using TensorFlow backend.


## Utils

In [3]:
## Convert lines for BERT
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    return np.array(all_tokens)

## Convert lines for BERT Cased rev dir
def convert_lines_rev(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[-max_seq_length:]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    return np.array(all_tokens)

# Convert lines for GPT
def convert_lines_gpt(example, max_seq_length,tokenizer):
    #max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(tokens_a)+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    #print(longer)
    return np.array(all_tokens)

def is_interactive():
    return 'SHLVL' not in os.environ

## Seeding
def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

## Read coeffs from pretrained word vectors 
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')

## Load Glove/Crawl
def load_embeddings(path):
    #with open(path,'rb') as f:
    emb_arr = KeyedVectors.load(path)
    return emb_arr

## Build embedding matrix
def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((max_features + 1, 300))
    unknown_words = []
    
    for word, i in word_index.items():
        if i <= max_features:
            try:
                embedding_matrix[i] = embedding_index[word]
            except KeyError:
                try:
                    embedding_matrix[i] = embedding_index[word.lower()]
                except KeyError:
                    try:
                        embedding_matrix[i] = embedding_index[word.title()]
                    except KeyError:
                        unknown_words.append(word)
    return embedding_matrix, unknown_words

## Sigmoid
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)    # (N, T, 1, K)
        x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
        x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T), some features are masked
        x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = x.squeeze(2)  # (N, T, K)
        return x

def handle_punctuation(x):
    x = x.translate(remove_dict)
    x = x.translate(isolate_dict)
    return x

def handle_contractions(x):
    x = tokenizer.tokenize(x)
    return x

def fix_quote(x):
    x = [x_[1:] if x_.startswith("'") else x_ for x_ in x]
    x = ' '.join(x)
    return x

def preprocess(x):
    x = handle_punctuation(x)
    x = handle_contractions(x)
    x = fix_quote(x)
    return x

class SequenceBucketCollator():
    def __init__(self, choose_length, sequence_index, length_index, label_index=None):
        self.choose_length = choose_length
        self.sequence_index = sequence_index
        self.length_index = length_index
        self.label_index = label_index
        
    def __call__(self, batch):
        batch = [torch.stack(x) for x in list(zip(*batch))]
        
        sequences = batch[self.sequence_index]
        lengths = batch[self.length_index]
        
        length = self.choose_length(lengths)
        mask = torch.arange(start=maxlen, end=0, step=-1) < length
        padded_sequences = sequences[:, mask]
        
        batch[self.sequence_index] = padded_sequences
        
        if self.label_index is not None:
            return [x for i, x in enumerate(batch) if i != self.label_index], batch[self.label_index]
    
        return batch

## LSTM Model

In [4]:
class ModelEmbedding(nn.Module):
    
    def __init__(self, max_features, embed_size, embedding_matrix, dropout = 0.3):
        
        super(ModelEmbedding, self).__init__()
        
        self.max_features = max_features
        self.embed_size = embed_size
        
        self.LUT = nn.Embedding(max_features, embed_size)
        self.LUT.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.LUT.weight.requires_grad = False
        self.LUT_dropout = SpatialDropout(dropout)
        
        #self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        h_embedding = self.LUT(x)
        h_embedding = self.LUT_dropout(h_embedding)
        
        return h_embedding

class NeuralNet(nn.Module):
    
    def __init__(self, embed_size, LSTM_UNITS, DENSE_HIDDEN_UNITS, num_aux_targets):
        super(NeuralNet, self).__init__()
        
        #embed_size = embedding_matrix.shape[1]
        
        #self.embedding = LUT
        
        # self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        # self.embedding.weight.requires_grad = False
        # self.embedding_dropout = SpatialDropout(0.3)
        
        self.lstm1 = nn.LSTM(embed_size, LSTM_UNITS, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(LSTM_UNITS * 2, LSTM_UNITS, bidirectional=True, batch_first=True)
    
        self.linear1 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        self.linear2 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        
        self.linear_out = nn.Linear(DENSE_HIDDEN_UNITS, 1)
        self.linear_aux_out = nn.Linear(DENSE_HIDDEN_UNITS, num_aux_targets)
        
    def forward(self, h_embedding, lengths=None):
        
        #h_embedding = self.embedding(x.long())
        #h_embedding = self.embedding_dropout(h_embedding)
        #set_trace()
        h_lstm1, _ = self.lstm1(h_embedding)
        h_lstm2, _ = self.lstm2(h_lstm1)
        
        # global average pooling
        avg_pool = torch.mean(h_lstm2, 1)
        # global max pooling
        max_pool, _ = torch.max(h_lstm2, 1)
        
        h_conc = torch.cat((max_pool, avg_pool), 1)
        h_conc_linear1  = F.relu(self.linear1(h_conc))
        h_conc_linear2  = F.relu(self.linear2(h_conc))
        
        hidden = h_conc + h_conc_linear1 + h_conc_linear2
        
        result = self.linear_out(hidden)
        aux_result = self.linear_aux_out(hidden)
        out = torch.cat([result, aux_result], 1)
        
        return out

class HyperBias(nn.Module):
    def __init__(self, LUT, NeuralNet):
        
        super(HyperBias, self).__init__()
        self.LUT = LUT
        self.Net = NeuralNet
        
    def forward(self, x, lengths=None):
        
        h_embedding = self.LUT(x.long())
        out = self.Net(h_embedding)
        
        return out

## Set up

In [5]:
## Set up
warnings.filterwarnings(action='once')
device = torch.device('cuda')

MAX_SEQUENCE_LENGTH = 300
SEED = 1234
BATCH_SIZE = 512
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

tqdm.pandas()
CRAWL_EMBEDDING_PATH = '../input/gensim-embeddings-dataset/crawl-300d-2M.gensim'
GLOVE_EMBEDDING_PATH = '../input/gensim-embeddings-dataset/glove.840B.300d.gensim'

if not is_interactive():
    def nop(it, *a, **k):
        return it

    tqdm = nop

    fastprogress.fastprogress.NO_BAR = True
    master_bar, progress_bar = force_console_behavior()
    fastai.basic_train.master_bar, fastai.basic_train.progress_bar = master_bar, progress_bar

seed_everything()

### Preprocess for BERT

In [6]:
def correct_spelling(x, dic):
    for word in dic.keys():
        if word in x:
            x = x.replace(word, dic[word])
    return x
###
def find_word(x,word):
    if word in x:
        return True
    else:
        return False
##
map_words = {'ᴀ':'A', 'ᴀɴᴅ':'AND', 'ᴛʜᴇ':'THE', 'Yᴏᴜ':'YOU', 'ʙʏ':'BY', 'ғᴏʀ':'FOR',
             'ᴄᴏᴍᴘᴜᴛᴇʀ':'COMPUTER', 'ᴛʜɪs':'THIS', 'ᴄʜᴇᴄᴋ':'CHECK', 'ᴡᴏʀᴋɪɴɢ':'WORKING',
             'ᴊᴏʙ':'JOB', 'ʜᴏᴜʀʟʏ':'HOURLY', 'ᴍᴏɴᴛʜ':'MONTH', 'ᴏғ':'OF', 'ᴛᴏ':'TO', 
             'ʟɪɴᴋ':'LINK', 'ᴜᴘ':'UP', 'ᴄᴀɴ':'CAN', 'ʜᴀᴠᴇ':'HAVE', 'Sᴛᴀʀᴛ':'START',
             'ғʀᴏᴍ':'FROM', 'ʜᴏᴍᴇ!':'HOME', 'Gʀᴇᴀᴛ':'GREAT', 'sᴛᴜᴅᴇɴᴛs,':'STUDENTS', 
             'sᴛᴀʏ-ᴀᴛ-ʜᴏᴍᴇ':'STAY AT HOME', 'ᴍᴏᴍs':'MOMS', 'ᴏʀ':'OR', 'ᴀɴʏᴏɴᴇ':'ANYONE',
             'ɴᴇᴇᴅɪɴɢ':'NEEDING', 'ᴀɴ':'AN', 'ᴇxᴛʀᴀ':'EXTRA', 'ᴏɴʟʏ':'ONLY', 'ɴᴇᴇᴅ':'NEED', 
            'ʀᴇʟɪᴀʙʟᴇ':'RELIABLE', 'ɪɴᴛᴇʀɴᴇᴛ':'INTERNET', 'ᴄᴏɴɴᴇᴄᴛɪᴏɴ...':'CONNECTION', 'Mᴀᴋᴇ':'MAKE',
            'ғᴏʟʟᴏᴡɪɴɢ':'FOLLOWING', 'ᴀᴛ':'AT', 'ʙᴏᴛᴛᴏᴍ':'BOTTOM', 'sɪɢɴɪɴɢ':'SIGNING', 'ᴜᴘ...':'UP', 
            'ʏᴏᴜʀ':'YOUR', 'ғɪʀsᴛ':'FIRST', 'ᴇɴᴅ':'END', 'ᴡᴇᴇᴋ...':'WEEK', 'ɪɴᴄᴏᴍᴇ...':'INCOME', 
             '𝒕𝒉𝒆':'THE', '𝒂𝒏𝒅':'AND', 'ᴍʏ':'MY', '𝒐𝒇':'OF', '𝒕𝒐':'TO',   '𝙩𝙝𝙚':'THE',
             '💙':'FRIENDSHIP', '❤️':'LOVE', '❤':'LOVE', '♡':'LOVE', '✔':'TICK',             
             '😂': 'JOY', '😂😂':'JOY', '😂😂😂':'CRY LAUGHING',  '💰':'MONEY',         
             '😁':'MISCHIEVOUS LAUGH', '😀':'HAPPY SMILE', '😉':'WINKING FACE','😃':'CHEERFUL SMILE',
             '🙄':'BORING', '😄':'JOY SMILE', '😊':'HAPPINESS', '😜':'FLIRT', '😎':'COOL',   
             '😆':'FUN HUMOUR', '👍':'APPROVAL', '🤔':'CONFUSED', '😅':'RELIEF', '☭':'COMMUNIST',
             '😡':'DISPLEASURE', '🤣':'FUNNY', '😢':'SAD', '☺':'SMILE',        
             '🙂':'FRIENDLY SMILE', '😇':'BLESSED',
             '😕':'CONFUSED', '😏':'WHIMSICAL FACE', '😳':'SHOCKED', '😝':'HUMOUR',
             '😲':'SHOCK', '😒':'SAD', '👍🏻':'APPROVAL', 
             '😠':'ANGRY', '😞':'SAD',  '😭':'VERY SAD',
             '😑':'NO WORDS',
             '😯':'WONDER','b!tch':'bitch', 'b1tch': 'bitch', 'd1ck':'dick','fcuk':'fuck','pussi':'pussy','sh!t':'shit','sh1t':'shit', 
             'Brexit': 'british exit'
            }

## BERT Part

In [7]:
BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)
bert_config = BertConfig('../input/bertconfig/bert_config.json')

### BERT with End words & Preprocess

In [8]:
import os
os.listdir('../input')

['endwordsepoch2',
 'tunedlossepoch2',
 'loss5epoch2',
 'bertmodel1extremeepoch2',
 'loss6epoch2',
 'ppbert',
 'endwordspreprocessepoch1',
 'jigsaw-unintended-bias-in-toxicity-classification',
 'bbe2s3',
 'gptpytorchoriginal',
 'bertlargeconfig',
 'bertloss4balancedepoch2',
 'loss4tunedepoch2',
 'casedm1extremeepoch2',
 'lstmmodeltesting',
 'bert-pretrained-models',
 'berttwoepochslr6',
 'gptfinal',
 'bertconfig',
 'gensim-embeddings-dataset',
 'gpt2-models',
 'casedconfig']

In [9]:
# Translate model from tensorflow to pytorch
MAX_SEQUENCE_LENGTH = 220
test_df = pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv")
test_df['comment_text'] = test_df['comment_text'].apply(lambda x: correct_spelling(x, map_words))
test_df['comment_text'] = test_df['comment_text'].astype(str) 
X_test = convert_lines_rev(test_df["comment_text"].fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH, tokenizer)

In [10]:
## BERT-3
model = BertForSequenceClassification(bert_config, num_labels=7)
model.load_state_dict(torch.load("../input/endwordsepoch2/bert_pytorch.bin"))
model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

## Make preds
test_preds = np.zeros((len(X_test)))
test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=512, shuffle=False)
tk0 = tqdm(test_loader)

for i, (x_batch,) in enumerate(tk0):
    #set_trace()
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds[i * 512:(i + 1) * 512] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred_bert_3 = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()

## BERT 1

In [11]:
### Read data
MAX_SEQUENCE_LENGTH = 250
test_df = pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv")
test_df['comment_text'] = test_df['comment_text'].apply(lambda x: correct_spelling(x, map_words))
test_df['comment_text'] = test_df['comment_text'].astype(str) 
X_test = convert_lines(test_df["comment_text"].fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH, tokenizer)

In [12]:
model = BertForSequenceClassification(bert_config, num_labels=7)
model.load_state_dict(torch.load("../input/bertmodel1extremeepoch2/bert_pytorch.bin"))
model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

# Make preds
test_preds = np.zeros((len(X_test)))
test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=512, shuffle=False)
tk0 = tqdm(test_loader)
for i, (x_batch,) in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds[i * 512:(i + 1) * 512] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred_bert_1 = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()

In [13]:
## BERT-2
model = BertForSequenceClassification(bert_config, num_labels=7)
model.load_state_dict(torch.load("../input/loss4tunedepoch2/bert_pytorch.bin"))
model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

## Make preds
test_preds = np.zeros((len(X_test)))
test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=512, shuffle=False)
tk0 = tqdm(test_loader)

for i, (x_batch,) in enumerate(tk0):
    #set_trace()
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds[i * 512:(i + 1) * 512] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred_bert_2 = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()

In [14]:
## BERT-4
model = BertForSequenceClassification(bert_config, num_labels=7)
model.load_state_dict(torch.load("../input/loss5epoch2/bert_pytorch.bin"))
model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

## Make preds
test_preds = np.zeros((len(X_test)))
test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=512, shuffle=False)
tk0 = tqdm(test_loader)

for i, (x_batch,) in enumerate(tk0):
    #set_trace()
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds[i * 512:(i + 1) * 512] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred_bert_4 = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()

### BERT Prediction

In [15]:
## Bert prediction
submission_bert = pd.DataFrame.from_dict({
    'id': test_df['id'],
    'prediction': 0.3*test_pred_bert_1 + 0.3*test_pred_bert_2 +  0.1*test_pred_bert_3 + 0.3*test_pred_bert_4
})

## BERT-Large

In [16]:
BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-24_h-1024_a-16/uncased_L-24_H-1024_A-16/'
bert_config = BertConfig('../input/bertlargeconfig/bert_config_large.json')

In [17]:
MAX_SEQUENCE_LENGTH = 220
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)
test_df = pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv")
test_df['comment_text'] = test_df['comment_text'].astype(str) 
X_test = convert_lines(test_df["comment_text"].fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH, tokenizer)

In [18]:
## BERT-Large
model = BertForSequenceClassification(bert_config, num_labels=7)
model.load_state_dict(torch.load("../input/bbe2s3/bert_large_pytorch.bin"))
model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

# Make preds
batch_bb = 256
test_preds = np.zeros((len(X_test)))
test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_bb, shuffle=False)
tk0 = tqdm(test_loader)
for i, (x_batch,) in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds[i * batch_bb:(i + 1) * batch_bb] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred_bert_big = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()

In [19]:
del model
gc.collect()

0

In [20]:
## Bert prediction
submission_bert_big = pd.DataFrame.from_dict({
    'id': test_df['id'],
    'prediction': test_pred_bert_big
})

## GPT

In [21]:
MAX_SEQUENCE_LENGTH = 220
GPT_MODEL_PATH = "../input/gpt2-models"
from pytorch_pretrained_bert import GPT2Tokenizer, GPT2ClassificationHeadModel
gpt_tokenizer = GPT2Tokenizer.from_pretrained(GPT_MODEL_PATH)
X_test = convert_lines_gpt(test_df["comment_text"].fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH, gpt_tokenizer)

/opt/conda/lib/python3.6/site-packages/pytorch_pretrained_bert/tokenization_gpt2.py:146: ResourceWarning: unclosed file <_io.TextIOWrapper name='../input/gpt2-models/vocab.json' mode='r' encoding='UTF-8'>
  self.encoder = json.load(open(vocab_file))
/opt/conda/lib/python3.6/site-packages/pytorch_pretrained_bert/tokenization_gpt2.py:151: ResourceWarning: unclosed file <_io.TextIOWrapper name='../input/gpt2-models/merges.txt' mode='r' encoding='utf-8'>
  bpe_data = open(merges_file, encoding='utf-8').read().split('\n')[1:-1]


In [22]:
model = GPT2ClassificationHeadModel.from_pretrained(GPT_MODEL_PATH, clf_dropout=0.4, n_class=7)
model.load_state_dict(torch.load("../input/gptfinal/gpt_pytorch.bin"))
model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

batch_size_gpt = 32
## Make preds
test_preds = np.zeros((len(X_test)))
test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size_gpt, shuffle=False)
tk0 = tqdm(test_loader)

for i, (x_batch,) in enumerate(tk0):
    #set_trace()
    pred = model(x_batch.to(device))
    test_preds[i * batch_size_gpt:(i + 1) * batch_size_gpt] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred_gpt = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()

In [23]:
del model
gc.collect()

0

In [24]:
## Bert prediction
submission_gpt = pd.DataFrame.from_dict({
    'id': test_df['id'],
    'prediction': test_pred_gpt
})

## LSTM Inference

In [25]:
symbols_to_isolate = '.,?!-;*"…:—()%#$&_/@＼・ω+=”“[]^–>\\°<~•≠™ˈʊɒ∞§{}·τα❤☺ɡ|¢→̶`❥━┣┫┗Ｏ►★©―ɪ✔®\x96\x92●£♥➤´¹☕≈÷♡◐║▬′ɔː€۩۞†μ✒➥═☆ˌ◄½ʻπδηλσερνʃ✬ＳＵＰＥＲＩＴ☻±♍µº¾✓◾؟．⬅℅»Вав❣⋅¿¬♫ＣＭβ█▓▒░⇒⭐›¡₂₃❧▰▔◞▀▂▃▄▅▆▇↙γ̄″☹➡«φ⅓„✋：¥̲̅́∙‛◇✏▷❓❗¶˚˙）сиʿ✨。ɑ\x80◕！％¯−ﬂﬁ₁²ʌ¼⁴⁄₄⌠♭✘╪▶☭✭♪☔☠♂☃☎✈✌✰❆☙○‣⚓年∎ℒ▪▙☏⅛ｃａｓǀ℮¸ｗ‚∼‖ℳ❄←☼⋆ʒ⊂、⅔¨͡๏⚾⚽Φ×θ￦？（℃⏩☮⚠月✊❌⭕▸■⇌☐☑⚡☄ǫ╭∩╮，例＞ʕɐ̣Δ₀✞┈╱╲▏▕┃╰▊▋╯┳┊≥☒↑☝ɹ✅☛♩☞ＡＪＢ◔◡↓♀⬆̱ℏ\x91⠀ˤ╚↺⇤∏✾◦♬³の｜／∵∴√Ω¤☜▲↳▫‿⬇✧ｏｖｍ－２０８＇‰≤∕ˆ⚜☁'
symbols_to_delete = '\n🍕\r🐵😑\xa0\ue014\t\uf818\uf04a\xad😢🐶️\uf0e0😜😎👊\u200b\u200e😁عدويهصقأناخلىبمغر😍💖💵Е👎😀😂\u202a\u202c🔥😄🏻💥ᴍʏʀᴇɴᴅᴏᴀᴋʜᴜʟᴛᴄᴘʙғᴊᴡɢ😋👏שלוםבי😱‼\x81エンジ故障\u2009🚌ᴵ͞🌟😊😳😧🙀😐😕\u200f👍😮😃😘אעכח💩💯⛽🚄🏼ஜ😖ᴠ🚲‐😟😈💪🙏🎯🌹😇💔😡\x7f👌ἐὶήιὲκἀίῃἴξ🙄Ｈ😠\ufeff\u2028😉😤⛺🙂\u3000تحكسة👮💙فزط😏🍾🎉😞\u2008🏾😅😭👻😥😔😓🏽🎆🍻🍽🎶🌺🤔😪\x08‑🐰🐇🐱🙆😨🙃💕𝘊𝘦𝘳𝘢𝘵𝘰𝘤𝘺𝘴𝘪𝘧𝘮𝘣💗💚地獄谷улкнПоАН🐾🐕😆ה🔗🚽歌舞伎🙈😴🏿🤗🇺🇸мυтѕ⤵🏆🎃😩\u200a🌠🐟💫💰💎эпрд\x95🖐🙅⛲🍰🤐👆🙌\u2002💛🙁👀🙊🙉\u2004ˢᵒʳʸᴼᴷᴺʷᵗʰᵉᵘ\x13🚬🤓\ue602😵άοόςέὸתמדףנרךצט😒͝🆕👅👥👄🔄🔤👉👤👶👲🔛🎓\uf0b7\uf04c\x9f\x10成都😣⏺😌🤑🌏😯ех😲Ἰᾶὁ💞🚓🔔📚🏀👐\u202d💤🍇\ue613小土豆🏡❔⁉\u202f👠》कर्मा🇹🇼🌸蔡英文🌞🎲レクサス😛外国人关系Сб💋💀🎄💜🤢َِьыгя不是\x9c\x9d🗑\u2005💃📣👿༼つ༽😰ḷЗз▱ц￼🤣卖温哥华议会下降你失去所有的钱加拿大坏税骗子🐝ツ🎅\x85🍺آإشء🎵🌎͟ἔ油别克🤡🤥😬🤧й\u2003🚀🤴ʲшчИОРФДЯМюж😝🖑ὐύύ特殊作戦群щ💨圆明园קℐ🏈😺🌍⏏ệ🍔🐮🍁🍆🍑🌮🌯🤦\u200d𝓒𝓲𝓿𝓵안영하세요ЖљКћ🍀😫🤤ῦ我出生在了可以说普通话汉语好极🎼🕺🍸🥂🗽🎇🎊🆘🤠👩🖒🚪天一家⚲\u2006⚭⚆⬭⬯⏖新✀╌🇫🇷🇩🇪🇮🇬🇧😷🇨🇦ХШ🌐\x1f杀鸡给猴看ʁ𝗪𝗵𝗲𝗻𝘆𝗼𝘂𝗿𝗮𝗹𝗶𝘇𝗯𝘁𝗰𝘀𝘅𝗽𝘄𝗱📺ϖ\u2000үսᴦᎥһͺ\u2007հ\u2001ɩｙｅ൦ｌƽｈ𝐓𝐡𝐞𝐫𝐮𝐝𝐚𝐃𝐜𝐩𝐭𝐢𝐨𝐧Ƅᴨןᑯ໐ΤᏧ௦Іᴑ܁𝐬𝐰𝐲𝐛𝐦𝐯𝐑𝐙𝐣𝐇𝐂𝐘𝟎ԜТᗞ౦〔Ꭻ𝐳𝐔𝐱𝟔𝟓𝐅🐋ﬃ💘💓ё𝘥𝘯𝘶💐🌋🌄🌅𝙬𝙖𝙨𝙤𝙣𝙡𝙮𝙘𝙠𝙚𝙙𝙜𝙧𝙥𝙩𝙪𝙗𝙞𝙝𝙛👺🐷ℋ𝐀𝐥𝐪🚶𝙢Ἱ🤘ͦ💸ج패티Ｗ𝙇ᵻ👂👃ɜ🎫\uf0a7БУі🚢🚂ગુજરાતીῆ🏃𝓬𝓻𝓴𝓮𝓽𝓼☘﴾̯﴿₽\ue807𝑻𝒆𝒍𝒕𝒉𝒓𝒖𝒂𝒏𝒅𝒔𝒎𝒗𝒊👽😙\u200cЛ‒🎾👹⎌🏒⛸公寓养宠物吗🏄🐀🚑🤷操美𝒑𝒚𝒐𝑴🤙🐒欢迎来到阿拉斯ספ𝙫🐈𝒌𝙊𝙭𝙆𝙋𝙍𝘼𝙅ﷻ🦄巨收赢得白鬼愤怒要买额ẽ🚗🐳𝟏𝐟𝟖𝟑𝟕𝒄𝟗𝐠𝙄𝙃👇锟斤拷𝗢𝟳𝟱𝟬⦁マルハニチロ株式社⛷한국어ㄸㅓ니͜ʖ𝘿𝙔₵𝒩ℯ𝒾𝓁𝒶𝓉𝓇𝓊𝓃𝓈𝓅ℴ𝒻𝒽𝓀𝓌𝒸𝓎𝙏ζ𝙟𝘃𝗺𝟮𝟭𝟯𝟲👋🦊多伦🐽🎻🎹⛓🏹🍷🦆为和中友谊祝贺与其想象对法如直接问用自己猜本传教士没积唯认识基督徒曾经让相信耶稣复活死怪他但当们聊些政治题时候战胜因圣把全堂结婚孩恐惧且栗谓这样还♾🎸🤕🤒⛑🎁批判检讨🏝🦁🙋😶쥐스탱트뤼도석유가격인상이경제황을렵게만들지않록잘관리해야합다캐나에서대마초와화약금의품런성분갈때는반드시허된사용🔫👁凸ὰ💲🗯𝙈Ἄ𝒇𝒈𝒘𝒃𝑬𝑶𝕾𝖙𝖗𝖆𝖎𝖌𝖍𝖕𝖊𝖔𝖑𝖉𝖓𝖐𝖜𝖞𝖚𝖇𝕿𝖘𝖄𝖛𝖒𝖋𝖂𝕴𝖟𝖈𝕸👑🚿💡知彼百\uf005𝙀𝒛𝑲𝑳𝑾𝒋𝟒😦𝙒𝘾𝘽🏐𝘩𝘨ὼṑ𝑱𝑹𝑫𝑵𝑪🇰🇵👾ᓇᒧᔭᐃᐧᐦᑳᐨᓃᓂᑲᐸᑭᑎᓀᐣ🐄🎈🔨🐎🤞🐸💟🎰🌝🛳点击查版🍭𝑥𝑦𝑧ＮＧ👣\uf020っ🏉ф💭🎥Ξ🐴👨🤳🦍\x0b🍩𝑯𝒒😗𝟐🏂👳🍗🕉🐲چی𝑮𝗕𝗴🍒ꜥⲣⲏ🐑⏰鉄リ事件ї💊「」\uf203\uf09a\uf222\ue608\uf202\uf099\uf469\ue607\uf410\ue600燻製シ虚偽屁理屈Г𝑩𝑰𝒀𝑺🌤𝗳𝗜𝗙𝗦𝗧🍊ὺἈἡχῖΛ⤏🇳𝒙ψՁմեռայինրւդձ冬至ὀ𝒁🔹🤚🍎𝑷🐂💅𝘬𝘱𝘸𝘷𝘐𝘭𝘓𝘖𝘹𝘲𝘫کΒώ💢ΜΟΝΑΕ🇱♲𝝈↴💒⊘Ȼ🚴🖕🖤🥘📍👈➕🚫🎨🌑🐻𝐎𝐍𝐊𝑭🤖🎎😼🕷ｇｒｎｔｉｄｕｆｂｋ𝟰🇴🇭🇻🇲𝗞𝗭𝗘𝗤👼📉🍟🍦🌈🔭《🐊🐍\uf10aლڡ🐦\U0001f92f\U0001f92a🐡💳ἱ🙇𝗸𝗟𝗠𝗷🥜さようなら🔼'

### Preprocessing

In [26]:
tokenizer = TreebankWordTokenizer()
isolate_dict = {ord(c):f' {c} ' for c in symbols_to_isolate}
remove_dict = {ord(c):f'' for c in symbols_to_delete}

x_test = test_df['comment_text'].progress_apply(lambda x:preprocess(x))
max_features = 410047

## Tokenization
tokenizer = text.Tokenizer(num_words = max_features, filters='',lower=False)
tokenizer.fit_on_texts(list(x_test))

100%|██████████| 97320/97320 [00:28<00:00, 3409.69it/s]


In [27]:
## Build embedding matrix on test set
crawl_matrix, unknown_words_crawl = build_matrix(tokenizer.word_index, CRAWL_EMBEDDING_PATH)
print('n unknown words (crawl): {}'.format(len(unknown_words_crawl)))

glove_matrix, unknown_words_glove = build_matrix(tokenizer.word_index, GLOVE_EMBEDDING_PATH)
print('n unknown words (glove): {}'.format(len(unknown_words_glove)))

max_features = max_features or len(tokenizer.word_index) + 1
print('max_features = {}'.format(max_features))

embedding_matrix = np.concatenate([crawl_matrix, glove_matrix], axis=-1)
embedding_matrix.shape

del crawl_matrix
del glove_matrix
gc.collect()

/opt/conda/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


n unknown words (crawl): 14412


/opt/conda/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


n unknown words (glove): 14704
max_features = 410047


3

In [28]:
# padding and convert comments to index sequence
x_test = tokenizer.texts_to_sequences(x_test)

maxlen = 300
test_lengths = torch.from_numpy(np.array([len(x) for x in x_test]))
x_test_padded = torch.from_numpy(sequence.pad_sequences(x_test, maxlen=maxlen))

In [29]:
batch_size = 512
test_dataset = data.TensorDataset(x_test_padded, test_lengths)
test_collator = SequenceBucketCollator(lambda lenghts: lenghts.max(), sequence_index=0, length_index=1)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=test_collator)

In [30]:
# Model Hyperparameters
NUM_MODELS = 2
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
MAX_LEN = 220
n_epochs = 5
n_aux_output=6
LSTM_PATH = '../input/lstmmodeltesting/'
all_test_preds = []
embed_size = embedding_matrix.shape[-1]
embed_size

600

In [31]:
# Placeholder for model predictions
model_test_preds = []

# LSTM inference
for model_idx in range(NUM_MODELS):
    
    all_test_preds = []
    checkpoint_weights = [2 ** epoch for epoch in range(n_epochs)]
 
    for epoch in range(n_epochs):
        
        print('Model: {}  Epoch: {}'.format(model_idx, epoch))
        LUT = ModelEmbedding(max_features, embed_size, embedding_matrix, dropout = 0.3)
        NET = NeuralNet(embed_size, LSTM_UNITS, DENSE_HIDDEN_UNITS, n_aux_output)
        file_path = 'model_' + str(model_idx) + '_' + str(epoch) + '.bin'
        model_path = os.path.join(LSTM_PATH, file_path)
        
        temp_dict = torch.load(model_path)
        NET.load_state_dict(temp_dict)
        
        model = HyperBias(LUT, NET)
        model.to(device)
        model.eval()
        
        test_preds = np.zeros((len(test_df), n_aux_output+1))
        for i, x_batch in enumerate(test_loader):
            #set_trace()
            X = x_batch[0].cuda()
            y_pred = sigmoid(model(X).detach().cpu().numpy())
            test_preds[i * batch_size:(i+1) * batch_size, :] = y_pred
        all_test_preds.append(test_preds)
        
        del model
        gc.collect()
        
    # if enable_checkpoint_ensemble:
    model_test_pred = np.average(all_test_preds, weights=checkpoint_weights, axis=0)
    model_test_preds.append(model_test_pred) 
#####
LSTM_pred = np.mean(model_test_preds, axis=0)[:, 0]

Model: 0  Epoch: 0
Model: 0  Epoch: 1
Model: 0  Epoch: 2
Model: 0  Epoch: 3
Model: 0  Epoch: 4
Model: 1  Epoch: 0
Model: 1  Epoch: 1
Model: 1  Epoch: 2
Model: 1  Epoch: 3
Model: 1  Epoch: 4


In [32]:
submission_lstm = pd.DataFrame.from_dict({
    'id': test_df['id'],
    'prediction': LSTM_pred
})

## Blending

In [33]:
submission = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv')
submission['prediction'] = 0.6*submission_bert.prediction + 0.15*submission_lstm.prediction + 0.10*submission_gpt.prediction+ 0.15*submission_bert_big.prediction
submission.to_csv('submission.csv', index=False)